# Drought Monitoring Using NDVI and EVI Using VNP13A1: VIIRS Vegetation Indices 16-Day 500m

Author - Nitin Magima

Date - Feb 2024

Version - 1.0

The jupyter notebook aims to generate seasonal anomaly maps for each n-day period of the rainy seasons in a country. It uses VIIRS product #13, Gridded Vegetation Indices (VNP13 Level 3 suite). The level 3 gridded vegetation indices are standard products designed to extend the significant VI time series derived from AVHRR and MODIS (Huete et al. 2002). The level 3 spatial and temporal gridded vegetation index products are composites of daily surface reflectances. They are generated at 500m, 1km, and 0.05o (~5.6km) every 8 days (quasi), 16 days, and calendar month. 

The standard Normalized Difference Vegetation Index (NDVI), referred to as the “continuity index” to the existing NOAA-AVHRR and MODIS derived NDVI. At the time of S-NPP launch S-NPP VIIRS VI User Guide-V2.1.1 8  (2011) there was nearly a 30-year NDVI record from AVHRR and MODIS (1981- and 2000-). VIIRS NDVI will extend this long term data record for use in operational monitoring studies.

The enhanced vegetation index (EVI) was developed to optimize the vegetation signal with improved sensitivity in high biomass regions and improved vegetation monitoring through a de- coupling of the canopy background signal and a reduction in atmosphere influences (Huete et al. 1997; Huete et al. 2002). 

Google Earth Engine (GEE) is a web platform for cloud-based processing of remote sensing data on a large scale. The advantage lies in its remarkable computation speed as processing is outsourced to Google servers. The platform provides a variety of constantly updated datasets; no download of raw imagery is required. While it is free of charge, one still needs to activate access to Google Earth Engine with a valid Google account.


DISCLAIMER

This is a set of scripts  shared for educational purposes only.  Anyone who uses this code or its
functionality or structure, assumes full liability and credits the author.

Map Disclaimer

The designations employed and the presentation of the material on this map do not imply the expression 
of any opinion whatsoever on the part of the author concerning the legal status of any country, territory, city or area or of its authorities, or concerning the delimitation of its 
frontiers or boundaries.

Sources
- [LANDDATAOPERATIONALPRODUCTSEVALUATION MODIS/VIIRS LAND PRODUCT QUALITY ASSESSMENT](https://landweb.modaps.eosdis.nasa.gov/browse?sensor=VIIRS&sat=SNPP)
- [Vegetation Index Product Suite User Guide & Abridged Algorithm Theoretical Basis Document](https://lpdaac.usgs.gov/documents/1372/VNP13_User_Guide_ATBD_V2.1.2.pdf)

# Google Earth Engine Setup

### Install Geemap

In [ ]:
%pip install -U "geemap[workshop]"

### Import libraries

Import the earthengine-api and geemap.

In [109]:
import ee
import geemap

### Authenticate and initialize Earth Engine

You will need to create a [Google Cloud Project](https://console.cloud.google.com/projectcreate) and enable the [Earth Engine API](https://console.cloud.google.com/apis/api/earthengine.googleapis.com) for the project. You can find detailed instructions [here](https://book.geemap.org/chapters/01_introduction.html#earth-engine-authentication).

In [110]:
ee.Authenticate()

True

Update the project below.

In [112]:
ee.Initialize(project="ee-training-412816")

## Initialize Map

In [113]:
# Creating a map
m = geemap.Map(basemap='WorldTopoMap')
m

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

# NDVI and EVI Calculation Setup

## Select Your Study Area

We use the [FAO GAUL: Global Administrative Unit Layers](https://data.apps.fao.org/catalog/dataset/global-administrative-unit-layers-gaul) for country focused analysis.

The Global Administrative Unit Layers (GAUL) compiles and disseminates the best available information on administrative units for all the countries in the world, providing a contribution to the standardization of the spatial dataset representing administrative units. The GAUL always maintains global layers with a unified coding system at country, first (e.g. departments), and second administrative levels (e.g. districts). Where data is available, it provides layers on a country by country basis down to third, fourth, and lowers levels.

You can use the drawing tools to draw a polygon on the map above as well.

Please enter the country name and admin level below. You can refer to the links below to help you. Use the "ADM0_NAME" column to use the correct name or spelling for a country.

1. [FAO GAUL: Global Administrative Unit Layers Admin 0 Reference](https://www.fao.org/in-action/countrystat/news-and-events/events/training-material/gaul-codes2014/en/)
2. [FAO GAUL: Global Administrative Unit Layers](https://data.apps.fao.org/catalog/dataset/gaul-codes)

In [114]:
country_name = 'Madagascar'
admin_level = 'level1' #use 'level0' or 'level1'

In [115]:
roi = m.user_roi

if roi is None:
    roi = ee.FeatureCollection(f"FAO/GAUL/2015/{admin_level}")
    roi = roi.filter(ee.Filter.eq('ADM0_NAME', country_name))
    
    # Define style parameters for visualization
    styleParams = {
        'fillColor': 'b5ffb4',
        'color': '00909F',
        'width': 1.0,
    }
    
    # Create a styled version of the ROI for visualization purposes only
    styledRoi = roi.style(**styleParams)
    
    # Add the styled ROI to the map for visualization
    m.addLayer(styledRoi, {}, country_name)

# Use 'roi' for clipping and other operations
# Do not use 'styledRoi' for operations other than visualization

# Check the type of 'roi', it should not return 'Image'
print('ROI type:', roi.getInfo()['type'])  # Should print 'FeatureCollection'

ROI type: FeatureCollection


## Load NDVI and EVI

In [116]:
# Load the VIIRS Vegetation Indices dataset.
viirs_dataset = ee.ImageCollection('NOAA/VIIRS/001/VNP13A1')

# Directly select the EVI, EVI2, and NDVI bands from the dataset.
evi_band = 'EVI'  # 3 band Enhanced Vegetation Index
evi2_band = 'EVI2'  # 2 band Enhanced Vegetation Index
ndvi_band = 'NDVI'  # Normalized Difference Vegetation Index

selected_bands = viirs_dataset.select([evi_band, evi2_band, ndvi_band])

## Visualizing Different Bands

### NDVI

In [117]:
# Visualize the NDVI band of the first image in the collection
first_image_ndvi = filtered_viirs.first().select(ndvi_band)
first_image_ndvi

In [118]:
# Clip the images to the Area of Interest
first_image_ndvi = first_image_ndvi.clip(roi)

# Set visualization parameters for NDVI.
ndvi_vis_params = {
  'min': 0,
  'max': 10000,
  'palette': ['000000', '004400', '008800', '00bb00', '00ff00'],
}

# Add the NDVI layer to the map using the specified visualization parameters.
m.addLayer(first_image_ndvi, ndvi_vis_params, 'NDVI')


# Display the map.
m

Map(bottom=2598.0, center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=Searc…

### EVI

In [67]:
# Visualize the NDVI band of the first image in the collection
first_image_evi = filtered_viirs.first().select(evi_band)
first_image_evi

In [68]:
# Clip the images to the Area of Interest
first_image_evi = first_image_evi.clip(roi)

# Set visualization parameters for EVI.
evi_vis_params = {
  'min': 0,
  'max': 10000,
  'palette': ['000000', '004400', '008800', '00bb00', '00ff00'],
}

# Add the NDVI layer to the map using the specified visualization parameters.
m.addLayer(first_image_evi, evi_vis_params, 'EVI')


# Display the map.
m

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

### Comparing NDVI vs EVI

In [69]:
left_layer = geemap.ee_tile_layer(first_image_evi, evi_vis_params, 'EVI')
right_layer = geemap.ee_tile_layer(first_image_ndvi, ndvi_vis_params, 'NDVI')

m.split_map(left_layer, right_layer)
m.center_object(roi, 5)
m

Map(center=[-19.335922158747607, 46.738223376282626], controls=(ZoomControl(options=['position', 'zoom_in_text…

# NDVI Calculations

## N-Day Anomaly Calculations

### Set Time Frame


In [120]:
firstimage = ee.Date(ee.List(selected_bands.get('date_range')).get(0))
latestimage = ee.Date(selected_bands.limit(1, 'system:time_start',  False).first().get('system:time_start'))

In [121]:
print(f"The observed time period for VIIRS begins on " + 
      f"{firstimage.format('YYYY-MM-dd').getInfo()} and ends on " + 
      f"{latestimage.format('YYYY-MM-dd').getInfo()}")

The observed time period for VIIRS begins on 2012-01-17 and ends on 2024-02-02


In [122]:
# Set the parameters

interval = 16 #Change it to the appropriate N-Day calculations you want to do
increment = 'day'
monitor_year = 2022 
monitor_month = 10 


# Define the time range you're interested in.
start = '2018-01-01'
end = f"{latestimage.format('YYYY-MM-dd').getInfo()}"

filtered_viirs = selected_bands.filterDate(start_date, end_date)

In [123]:
# Convert the start date string to an ee.Date object
startDate = ee.Date(start)

# Advance the start date by the interval and convert to milliseconds
secondDate = startDate.advance(interval, increment).millis()

# Calculate the increase in milliseconds between the start and second date
increase = secondDate.subtract(startDate.millis())

# Generate a list of milliseconds from the start date to the end date, by the calculated increment
decad_list = ee.List.sequence(startDate.millis(), ee.Date('2019-01-01').millis(), increase)

This 'decad_list' contains the starting milliseconds for each period in the sequence. This list is used to map over each date, applying a function that filters the VIIRS ImageCollection to the specific decadal period, then computes the mean and standard deviation for that period.

ee.Filter.calendarRange is used to filter the modis ImageCollection to images within each decadal period specified by the start and end days of the year. The .set method is used to add a property ('day_of_year') to each resulting image, indicating the start day of the year for the decadal period.

In [124]:
# Select the NDVI band

ndvi = filtered_viirs.select(ndvi_band)

# Calculate mean images for each decadal period

means = ee.ImageCollection(decad_list.map(lambda date: ndvi.filter(
    ee.Filter.calendarRange(
        ee.Date(date).getRelative('day', 'year'), 
        ee.Date(date).advance(interval, increment).getRelative('day', 'year')
    )
).mean().set('day_of_year', ee.Date(date).getRelative('day', 'year'))))

# Calculate standard deviation images for each decadal period
std_devs = ee.ImageCollection(decad_list.map(lambda date: ndvi.filter(
    ee.Filter.calendarRange(
        ee.Date(date).getRelative('day', 'year'), 
        ee.Date(date).advance(interval, increment).getRelative('day', 'year')
    )
).reduce(ee.Reducer.stdDev()).set('day_of_year', ee.Date(date).getRelative('day', 'year'))))

This code snippet sets a time range to run through, calculates the latest data date, and generates a list of decadal (or specified interval) dates within the window from the start date to the minimum of the window's maximum end date or the latest data date available.

- The reduceColumns function is used to get the minimum and maximum of the "system:time_start" property in the NDVI dataset, identifying the latest data date.
- The window's start and end dates are defined to set the period you're interested in. The end date is the minimum of the window's maximum end date or the latest data available in the dataset.
- The ee.List.sequence function generates a list of dates (in milliseconds since the epoch) from the window's start date to its end date, stepping by the specified interval and increment.

In [125]:
# Set time range to run through
windowStartDate = ee.Date.fromYMD(monitor_year, monitor_month, 1)
windowMaxEndDate = windowStartDate.advance(90, 'day')  # This is the absolute latest date

# Get the range of dates in the dataset
range = ndvi.reduceColumns(ee.Reducer.minMax(), ["system:time_start"])
latestDataDate = ee.Date(range.get('max'))

# Calculate the window end date as the minimum of windowMaxEndDate and latestDataDate
windowEndDate = ee.Date(ee.Number(windowMaxEndDate.millis()).min(ee.Number(latestDataDate.millis())))

# Calculate the second window date and the increase in milliseconds
secondWindowDate = windowStartDate.advance(interval, increment).millis()
windowIncrease = secondWindowDate.subtract(windowStartDate.millis())

# Generate a list of dates (milliseconds) from windowStartDate to windowEndDate by the calculated increase
window_decad_list = ee.List.sequence(windowStartDate.millis(), windowEndDate.millis(), windowIncrease)

The following code focuses on the process of grouping images by decad (a ten-day period or a custom definition based on your interval), then reducing within those groups by the mean. This process involves creating an ImageCollection where each image represents the mean of images within each group, adjusted by subtracting the mean and dividing by the standard deviation for the same period.

In [129]:
# Function to process each date in the list
def process_date(date):
    '''
    The process_date function calculates the "decad" value by rounding the day of the year to the nearest 
    interval you're interested in. For each date, it filters the NDVI ImageCollection to images within the 
    specified range, computes their mean, and then adjusts this mean by subtracting the precomputed mean and 
    dividing by the precomputed standard deviation for the same interval.

    '''
    date = ee.Date(date)
    decad = date.getRelative('day', 'year').divide(16).round().multiply(16)  # Adjust the rounding as needed

    # Filter the NDVI collection within the date range and calculate the mean
    mean = ndvi.filterDate(date, date.advance(interval, increment)) \
                .mean() \
                .set('day_of_year', decad)
    
    # Subtract the overall mean and divide by the standard deviation for the same period
    # means and std_devs are assumed to be ImageCollections where each image is annotated 
    # with a day_of_year property corresponding to its decad. They are used to normalize 
    # the computed mean for each period.
    adjusted_mean = mean.subtract(
        means.filter(ee.Filter.eq('day_of_year', decad)).first()
    ).divide(
        std_devs.filter(ee.Filter.eq('day_of_year', decad)).first()
    ).set({
        'day_of_year': decad,
        'date': date.millis()
    })
    
    return adjusted_mean

# Apply the function to each date in the window_decad_list to create a new ImageCollection

bydecad = ee.ImageCollection.fromImages(window_decad_list.map(process_date))

# byDecad, is an ImageCollection with one image per period, each adjusted by the corresponding 
# period's mean and standard deviation.

In [130]:
# Clip all images in the ImageCollection to the roi
bydecad_clipped = bydecad.map(lambda image: image.clip(roi))

vis_params = {
    'min': -3.0,
    'max': 3.0,
    'palette': ['a36437', 'FCD163', '207401'],
}

# Add the NDVI layer to the map using the specified visualization parameters.
# Since byDecad_clipped is an ImageCollection, you cannot directly add it as a layer.
# You could add specific images, or composite the collection. Here, we'll add the first image as an example:
first_image = bydecad_clipped.first()

m.addLayer(first_image, vis_params, 'NDVI Anomaly Analysis')
#m.add_time_slider(images, vis_params, time_interval=2)

# Center the map around the region of interest
m.centerObject(roi)

# Display the map.
m

Map(bottom=18314.0, center=[-18.778447291809044, 46.840154327781306], controls=(WidgetControl(options=['positi…